In [6]:
import glob
import pandas as pd
import os
import glob
from datetime import datetime,timedelta

# Personnal Import 
from utilities import get_mode_date2path
from load_data import load_subway_15_min,load_CRITER,load_netmob

## Dataset : Metro 15 minutes 2019 2020
- On va travailler sur les mois de Mars, Avril, Mai 2019, avec les 4 lignes de Metro A, B, C, D ainsi que la ligne de Funiculaire '326'.
- On intègrera les données NetMob dans la suite.

In [7]:
try : 
    df_metro = pd.read_csv('data/Metro_15min_mar2019_mai2019.csv',index_col = 0)
    df_metro.datetime = pd.to_datetime(df_metro.datetime)

except:
    folder_path = '../../Data/keolis_data_2019-2020/'
    txt_path = "Métro 15 minutes 2019 2020.txt"
    start,end = datetime(2019,3,16),datetime(2019,6,1)

    df_metro_funi_2019_2020 = load_subway_15_min(folder_path+txt_path)
    df_metro = df_metro_funi_2019_2020[(df_metro_funi_2019_2020.datetime >= start)&(df_metro_funi_2019_2020.datetime < end)]

df_metro.head()

,datetime,Station,Code ligne,in,out
288362,2019-03-16 00:00:00,Ampère Victor Hugo,A,34,30.0
288363,2019-03-16 00:15:00,Ampère Victor Hugo,A,40,34.0
288364,2019-03-16 00:30:00,Ampère Victor Hugo,A,17,14.0
288365,2019-03-16 00:45:00,Ampère Victor Hugo,A,1,16.0
288366,2019-03-16 01:00:00,Ampère Victor Hugo,A,0,0.0


## Build Feature Vector : 

## Build Adjacency Matrix : 

## Load Models and Train 
- Essaye de capturer des dépendances spatio-temporelle. 
- Créer un modèle avec un module d'attention ou Transformer

## Ajout des données NetMob

## Concentration sur les événements atypiques / anomalies